# 75ML30D
![17%](https://progress-bar.dev/17?title=progress&width=100)

#### Description:
- Solving 75 Medium Leetcode problems in 30 days.<br>
- Languange of choice: `Python`<br>

#### Timelines:
- Start Date: 9th Jan 2024
- Challenge End Date: 8th Feb 2024

In [1]:
from typing import List
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    return temp
    
conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)
        return visited

    def __str__(self):
        return str(self.val)

    def from_array(nums: List[int]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)
                
                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

temp = BinaryTreeNode.from_array([1,2,3,4,5,6,None,None,None,7,8])
temp.to_list()

[1, 2, 3, 4, 5, 6, None, None, None, 7, 8, None, None, None, None, None, None]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr
            
        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

# 1. 
Date: 09th Jan 2024<br>
Medium Leetcode: minimum-path-sum<br>
Solution: https://leetcode.com/problems/minimum-path-sum/submissions/1141069624

In [4]:
class Solution:
    def minPathSum(self, grid: List[List[int]]) -> int:
        # Compute once
        height = len(grid)
        width = len(grid[0])

        # Start at the bottom right
        x, y = height - 1, width - 1

        while x >= 0:
            while y >= 0:
                
                add = min(
                    grid[x][y + 1] if y + 1 < width else math.inf,
                    grid[x + 1][y] if x + 1 < height else math.inf
                )
                
                if add != math.inf:
                    grid[x][y] += add
                    
                # Decrement y (go left)
                y -= 1

            # Decrement x (go up)
            x -= 1

            # Reset y
            y = width - 1

        return grid[0][0]

# Test the solution
assert Solution().minPathSum([[1,3,1],[1,5,1],[4,2,1]]) == 7
assert Solution().minPathSum([[1,2,3],[4,5,6]]) == 12

# 2. 
Date: 10th Jan 2024<br>
Medium Leetcode: amount-of-time-for-binary-tree-to-be-infected<br>
Solution: https://leetcode.com/problems/amount-of-time-for-binary-tree-to-be-infected/submissions/1141991104

In [5]:
class Solution:
    def amountOfTime(self, root: BinaryTreeNode, start: int) -> int:
        matrix = dict()
        def getAdjacencyMatrix(curr, parent=None):
            if curr:
                vertices = matrix.get(curr, [])

                if parent:
                    vertices.append(parent.val)
                if curr.right:
                    vertices.append(curr.right.val)
                if curr.left:
                    vertices.append(curr.left.val)

                matrix[curr.val] = vertices
    
                # Recurse
                getAdjacencyMatrix(curr.left, curr)
                getAdjacencyMatrix(curr.right, curr)
            

        getAdjacencyMatrix(root)

        # Do a BFS until all nodes are exhausted
        minute = 0
        infected = set({start})
        neighbours = matrix[start]

        while neighbours:
            next_neighbours = []
            for node in neighbours:
                if node not in infected:
                    infected.add(node)
                    next_neighbours.extend(list(filter(lambda x: x not in infected, matrix[node])))

            neighbours = next_neighbours
            minute += 1
                
        return minute
        
# Test the solution
assert Solution().amountOfTime(BinaryTreeNode.from_array([1,5,3,None,4,10,6,9,2]), 3) == 4
assert Solution().amountOfTime(BinaryTreeNode.from_array([1]), 1) == 0

# 3. 
Date: 10th Jan 2024<br>
Medium Leetcode: dota2-senate<br>
Solution: https://leetcode.com/problems/dota2-senate/submissions/1142503920/

In [6]:
class Solution:
    def predictPartyVictory(self, senate: str) -> str:

        while senate:

            result = []
            count = 0
            for ch in senate:
                if count == 0:
                    result.append(ch)
                    count = 1
                elif ch != result[-1]:
                    count -= 1
                else:
                    result.append(ch)
                    count += 1
            
            new_result = []
            for ch in result:
                if count > 0 and ch != result[-1]:
                    count -= 1
                else:
                    new_result.append(ch)
    
            senate = "".join(new_result)

            if count == len(senate):
                return "Radiant" if senate[0] == "R" else "Dire"

# Test the solution
assert Solution().predictPartyVictory("RDRDDDRDDRDRDDRRRRRD") == "Dire"
assert Solution().predictPartyVictory("RDRDDRD") == "Dire"
assert Solution().predictPartyVictory("RRDDD") == "Radiant"

# 4. 
Date: 11th Jan 2024<br>
Medium Leetcode: populating-next-right-pointers-in-each-node<br>
Solution: https://leetcode.com/problems/populating-next-right-pointers-in-each-node/submissions/1142856094/

In [7]:
class Solution:
    def connect(self, root):
        'This takes O(log(n)) space and O(1) complexity, q asked for O(1) space' 

        def backtrack(curr: Node, level: int = 0, right_most: dict = {}):
            if not curr:
                return 

            if level in right_most:
                right_most[level].next = curr

            right_most[level] = curr
            backtrack(curr.left, level + 1, right_most)
            backtrack(curr.right, level + 1, right_most)

        backtrack(root)
        return root

# 5. 
Date: 11th Jan 2024<br>
Medium Leetcode: maximum-difference-between-node-and-ancestor<br>
Solution: https://leetcode.com/problems/maximum-difference-between-node-and-ancestor/submissions/1142882452

In [8]:
class Solution:
    def maxAncestorDiff(self, root: BinaryTreeNode) -> int:
        
        def backtrack(curr):
            
            if not curr:
                return None, None, None

            lmin, lmax, ldiff = backtrack(curr.left) 
            rmin, rmax, rdiff = backtrack(curr.right)
            currMin = currMax = curr.val
            maxDiff = -math.inf

            if curr.left:
                currMin = min(currMin, lmin)
                currMax = max(currMax, lmax)
                maxDiff = max(
                    maxDiff,
                    abs(curr.val - lmin),
                    abs(curr.val - lmax),
                    ldiff
                )

            if curr.right:
                currMin = min(currMin, rmin)
                currMax = max(currMax, rmax)
                maxDiff = max(
                    maxDiff,
                    abs(curr.val - rmin),
                    abs(curr.val - rmax),
                    rdiff
                )

            return currMin, currMax, maxDiff

        return backtrack(root)[2]

# Test ths solution
assert Solution().maxAncestorDiff(BinaryTreeNode.from_array([8,3,10,1,6,None,14,None,None,4,7,13])) == 7
assert Solution().maxAncestorDiff(BinaryTreeNode.from_array([8,3,10,1,6,None,14,None,None,50,7,13])) == 47
assert Solution().maxAncestorDiff(BinaryTreeNode.from_array([1,None,2,None,0,3])) == 3

# 6. 
Date: 11th Jan 2024<br>
Medium Leetcode: delete-the-middle-node-of-a-linked-list<br>
Solution: https://leetcode.com/problems/delete-the-middle-node-of-a-linked-list/submissions/1142953098

In [9]:
class Solution:
    def deleteMiddle(self, head: ListNode) -> ListNode:
        'Better approach after seeing the solution is to use the fast pointer / slow pointer approach'
        n, curr = 0, head  
        while curr:
            curr = curr.next
            n += 1

        mid = n // 2

        if mid == 0:
            return None

        prev, curr = None, head
        while mid > 0:
            prev, curr = curr, curr.next
            mid -= 1

        prev.next = curr.next
        curr.next = None
        return head

# Test the solution
assert Solution().deleteMiddle(ListNode.to_singly_linked_list([1])) is None
assert Solution().deleteMiddle(ListNode.to_singly_linked_list([1, 2])).to_list() == [1]
assert Solution().deleteMiddle(ListNode.to_singly_linked_list([1, 2, 3])).to_list() == [1, 3]
assert Solution().deleteMiddle(ListNode.to_singly_linked_list([1, 2, 3, 4])).to_list() == [1, 2, 4]

# 7. 
Date: 12th Jan 2024<br>
Medium Leetcode: find-minimum-in-rotated-sorted-array<br>
Solution: https://leetcode.com/problems/find-minimum-in-rotated-sorted-array/submissions/1144294358/

In [10]:
class Solution:
    def findMin(self, nums: List[int]) -> int:
        """
        Objective is to find i such that nums[i] < nums[i - 1]
        """

        # O(1) time complexity
        n = len(nums)

        # Edge case - 1
        if n == 1:
            return nums[0]

        # Edge case - 2
        elif nums[0] < nums[-1]:
            return nums[0]
        
        else:
            start, end = 1, n - 1
            while start <= end:
                mid = (start + end) // 2

                if nums[mid] < nums[mid - 1]:
                    return nums[mid]
                
                elif nums[mid] > nums[0]:
                    start = mid + 1
                
                else:
                    end = mid - 1
            return -1

# Test the solution
assert Solution().findMin([4,5,6,7,0,1,2]) == 0
assert Solution().findMin([11,13,15,17]) == 11
assert Solution().findMin([2,3,5,6,7,1]) == 1
assert Solution().findMin([2,3,5,6,7,8,1]) == 1
assert Solution().findMin([6,7,8,1,2,3,4,5]) == 1

# 8. 
Date: 12th Jan 2024<br>
Medium Leetcode: count-and-say<br>
Solution: https://leetcode.com/problems/count-and-say/submissions/1144310034/

In [11]:
class Solution:
    def countAndSay(self, n: int) -> str:
        def say(s: str):
            prev = None
            count = 0
            result = []
            for ch in s + " ":
                if prev != ch:
                    if prev:
                        result.append(f"{count}{prev}")
                    prev, count = ch, 1
                else:
                    count += 1
                    
            return "".join(result)

        curr = "1" 
        while n > 1:
            curr = say(curr)
            n -= 1

        return curr

# Test the solution
assert Solution().countAndSay(10) == "13211311123113112211"
assert Solution().countAndSay(4) == "1211"

# 9. 
Date: 13th Jan 2024<br>
Medium Leetcode: minimum-number-of-steps-to-make-two-strings-anagram<br>
Solution: https://leetcode.com/problems/minimum-number-of-steps-to-make-two-strings-anagram/submissions/1144703174

In [12]:
class Solution:
    def minSteps(self, s: str, t: str) -> int:
        currentCounter = collections.Counter(s)
        targetCounter = collections.Counter(t)
        
        count = 0
        for k, v in currentCounter.items():
            count += max(v - targetCounter.get(k, 0), 0)
        
        return count
    
# Test the solution
assert Solution().minSteps("leetcode", "practice") == 5
assert Solution().minSteps("anagram", "mangaar") == 0
assert Solution().minSteps("bab", "aba") == 1

# 10. 
Date: 13th Jan 2024<br>
Medium Leetcode: longest-common-subsequence<br>
Solution: https://leetcode.com/problems/longest-common-subsequence/submissions/1144816986

In [13]:
class Solution:
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        
        n1, n2 = len(text1), len(text2)

        def iterate() -> int:
            prev = [0 for _ in range(n1 + 1)]
            for i in range(n2 - 1, -1, -1):
                curr = [0 for _ in range(n1 + 1)]
                for j in range(n1 - 1, -1, -1):
                    if text1[j] == text2[i]:
                        curr[j] = 1 + prev[j + 1]
                    else:
                        curr[j] = max(prev[j], curr[j + 1])
                prev = curr
                        
            return curr[0]

        @functools.cache
        def backtrack(i: int, j: int) -> int:
            if i == n1 or j == n2:
                return 0
            elif text1[i] == text2[j]:
                return 1 + backtrack(i + 1, j + 1)
            else:
                return max(backtrack(i, j + 1), backtrack(i + 1, j))
        
        return iterate()

# Testing the solution
assert Solution().longestCommonSubsequence("abcded", "acede") == 4
assert Solution().longestCommonSubsequence("", "acede") == 0

# 11. 
Date: 13th Jan 2024<br>
Medium Leetcode: edit-distance<br>
Solution: https://leetcode.com/problems/edit-distance/submissions/1145197901/

In [14]:
class Solution:
    def minDistance(self, word1: str, word2: str) -> int:
        """
        Remove / insert characters towards the end to match length
        Replace all characters with characters from word2
        Upper bound = max(n1, n2) - min(n1, n2) + n2

        # Optimized approach after watching 2 min of neetcode - watch tutorial
        """

        # Find the length of both the words
        n1, n2 = len(word1), len(word2) 

        @functools.cache
        def backtrack(i: int, j: int) -> int:
            if i >= n1 or j >= n2:
                return max(n1 - i, n2 - j)
            elif word1[i] == word2[j]:
                return backtrack(i + 1, j + 1)
            else:
                return 1 + min(
                    # Replace current char
                    backtrack(i + 1, j + 1),

                    # Remove current char
                    backtrack(i + 1, j),

                    # Insert char from target string
                    backtrack(i, j + 1)
                )
        
        answer = backtrack(0, 0)
        return answer

# Testing the solution
assert Solution().minDistance("horse", "ros") == 3
assert Solution().minDistance("intention", "execution") == 5
assert Solution().minDistance("mississippi", "missouri") == 6
assert Solution().minDistance("", "abcd") == 4
assert Solution().minDistance("abcd", "") == 4
assert Solution().minDistance("", "") == 0

# 12. 
Date: 14th Jan 2024<br>
Medium Leetcode: determine-if-two-strings-are-close<br>
Solution: https://leetcode.com/problems/determine-if-two-strings-are-close/submissions/1145505023

In [15]:
class Solution:
    def closeStrings(self, word1: str, word2: str) -> bool:
        n1 = len(word1)
        n2 = len(word2) 

        if n1 != n2:
            return False
        
        else:
            counter1 = collections.Counter(word1)
            counter2 = collections.Counter(word2)
            freq_match = sorted(counter1.values()) == sorted(counter2.values())
            key_match = sorted(counter1.keys()) == sorted(counter2.keys())
            return freq_match and key_match

# Test the solution
assert Solution().closeStrings("abc", "bca") == True
assert Solution().closeStrings("a", "aa") == False
assert Solution().closeStrings("cabbba", "abbccc") == True
assert Solution().closeStrings("faddda", "dccaaa") == False
assert Solution().closeStrings("faddda", "dffaaa") == True
assert Solution().closeStrings("faddda", "dffdaa") == False
assert Solution().closeStrings("aaabbbbccddeeeeefffff", "aaaaabbcccdddeeeeffff") == False

# 13. 
Date: 14th Jan 2024<br>
Medium Leetcode: find-beautiful-indices-in-the-given-array-i<br>
Solution: https://leetcode.com/problems/find-beautiful-indices-in-the-given-array-i/submissions/1145567672/

In [16]:
class Solution:
    def beautifulIndices(self, s: str, a: str, b: str, k: int) -> List[int]:
        result = []
        for i, ch in enumerate(s):
            if ch == a[0] and a == s[i: i + len(a)]:
                if b in s[max(i - k, 0): i + k + len(b)]:
                    result.append(i)
        
        return result

# Test the solution
assert Solution().beautifulIndices("abcd", "a", "b", 1) == [0]
assert Solution().beautifulIndices("abcd", "a", "cd", 1) == []
assert Solution().beautifulIndices("a", "a", "a", 1) == [0]
assert Solution().beautifulIndices("a", "a", "aa", 1) == []
assert Solution().beautifulIndices("vatevavakz", "va", "lbda", 1) == []